In [38]:
import pytesseract
from pytesseract import Output
import cv2
import pandas as pd
import numpy as np
import os

In [39]:
image_path = 'C:/Users/aryan/Desktop/WORK/business quant/[Business Quant] Image dataset for OCR/image00004.png'

In [57]:
#img is read
image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

In [59]:
#detect structure using opencv
horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (25, 1))
vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1, 25))
horizontal_lines = cv2.morphologyEx(image, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
vertical_lines = cv2.morphologyEx(image, cv2.MORPH_OPEN, vertical_kernel, iterations=2)

In [60]:
#structure is defined
table_structure = cv2.add(horizontal_lines, vertical_lines)

In [61]:
#bounding boxes are found
contours, _ = cv2.findContours(table_structure, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
bounding_boxes = [cv2.boundingRect(c) for c in contours]

In [62]:
bounding_boxes = sorted(bounding_boxes, key=lambda x: x[1])

In [74]:
#path to output
output_csv_path = os.path.expanduser('~/ocr_output3.csv')

In [64]:
ocr_data = []

In [68]:
#iterate through bounding boxes to recognize text
for i, (x, y, w, h) in enumerate(bounding_boxes):
    roi = image[y:y+h, x:x+w]
    #tesseract is used for ocr
    ocr_result = pytesseract.image_to_data(roi, output_type=Output.DICT)
    
    for j in range(len(ocr_result['text'])):
        text = ocr_result['text'][j].strip()
        if text:  
            row_number = i  
            col_number = ocr_result['left'][j] 
            ocr_data.append({'Row': row_number, 'Col': col_number, 'Text': text})

In [69]:
#list to dataframe
df = pd.DataFrame(ocr_data)

In [72]:
#dataframe is sorted
if not df.empty:
    df = df.sort_values(by=['Row', 'Col'])

In [75]:
#save as csv file
df.to_csv(output_csv_path, index=False, header=False)